# Construyendo buenos agentes

*Este tutorial es una adaptación de: https://huggingface.co/docs/smolagents/tutorials/building_good_agents *

Como hemos visto en la guía de inicio rápido, la construcción de agentes se aleja bastante de la programación explícita y determinista a la que podemos estar aconstrumbrados como programadores. Por lo tanto, existe un cierto riesgo de que construyamos un agente con una intención, pero que finalmente dicho agente no sea efectivo a la hora de realizar las acciones deseadas.

Uno de los factores clave para construir buenos antes es la simplicidad, la obtendremos simplificando el flujo de trabajo de nuestro agente tanto como sea posible. Si dejamos más margen para que el LLM tenga que aportar soluciones explícitas a problemas relativamente complejos, existe un riesgo mayor de que el agente no funcione como deseamos.

Imaginemos, por ejemplo, que queremos construir una web de viajes de Surf basada en agentes, una posible aproximación sería tener dos herramientas distintas que accediesen a un API de mapas para calcular la distancia hasta un punto determinado, y a un API del tiempo para conocer qué tipo va a hacer próximamente en esa localización. También, podríamos simplificar este flujo de trabajo encapsulando en una única herramienta *return_spot_information* el acceso a ambas API devolviendo una salida que concatena las dos informaciones. Estos nos proporciona dos enseñanzas:

- Debemos reducir el número de herramientas que podemos usar, por ejemplo, consolidando dos herramientas en una, si sabemos que dichas herramientas se van a usar siempre a la vez.
- También, la lógica del agente se debería basar cuando sea posible en funciones deterministas, en vez de en decisiones del agente. Por lo tanto, debemos usar el LLM sólo cuando sea indispensable, y no podamos programar una lógica determinista que exprese de forma precisa cómo tomar una decisión.

## Mejora el flujo de información hacia el LLM

Debemos recordar que el agente solo recibe información del exterior, o lo puede manipular, a través del conjunto de herramientas que tiene disponible. Por ello, debemos asegurarnos que las herramientas registran toda la información que podría ser útil al motor LLM de forma explícita y lo más precisa posible. 

Vamos a empezar con una herramienta simple que recupera información sobre el tiempo basándose en la localización y la fecha y hora. Empecemos por una primera aproximación.

In [1]:
import datetime
from smolagents import tool

def get_weather_report_at_coordinates(coordinates, date_time):
    # Dummy function, returns a list of [temperature in °C, risk of rain on a scale 0-1, wave height in m]
    return [28.0, 0.35, 0.85]

def convert_location_to_coordinates(location):
    # Returns dummy coordinates
    return [3.3, -42.0]

@tool
def get_weather_api(location: str, date_time: str) -> str:
    """
    Returns the weather report.

    Args:
        location: the name of the place that you want the weather for.
        date_time: the date and time for which you want the report.
    """
    lon, lat = convert_location_to_coordinates(location)
    date_time = datetime.strptime(date_time)
    return str(get_weather_report_at_coordinates((lon, lat), date_time))

Observemos varios errores en esta definición:

- El formato de la información sobre el *date_time* no está expresado de forma precisa, y esta podría ser una posible fuente de error.
- No se especifica de forma precisa cómo se debe proporcionar la localización a consultar.
- No hay mecanismos de registro (*logging*) de errores explícitos en la especificación de la localización o en el formato de la fecha y hora.
- El formato de salida es difícil de entender.

Aun así, si la llamada a la herramienta falla, la traza del error que se produce, podría ayudar al LLM a hacer ingeniería inversa para ser capaz a través de varias iteraciones de prueba y error, de resolver los problemas que se vayan presentado. Sin embargo, esto aumenta el número de iteraciones que el LLM requiere para alcanzar una llamada a la herramienta que resulte efectiva, el proceso se alarga, requiere más llamadas al LLM, lo cuál puede aumentar el coste económico (por ejemplo, si usamos un API de LLM de pago por uso).

Por ello, vamos a ver una definición alternativa de esta herramienta que nos permita subsanar alguna de las deficiencias detectadas en la versión inicial.


In [2]:
@tool
def get_weather_api(location: str, date_time: str) -> str:
    """
    Returns the weather report.

    Args:
        location: the name of the place that you want the weather for. Should be a place name, followed by possibly a city name, then a country, like "Anchor Point, Taghazout, Morocco".
        date_time: the date and time for which you want the report, formatted as '%m/%d/%y %H:%M:%S'.
    """
    lon, lat = convert_location_to_coordinates(location)
    try:
        date_time = datetime.strptime(date_time)
    except Exception as e:
        raise ValueError("Conversion of `date_time` to datetime format failed, make sure to provide a string in format '%m/%d/%y %H:%M:%S'. Full trace:" + str(e))
    temperature_celsius, risk_of_rain, wave_height = get_weather_report_at_coordinates((lon, lat), date_time)
    return f"Weather report for {location}, {date_time}: Temperature will be {temperature_celsius}°C, risk of rain is {risk_of_rain*100:.0f}%, wave height is {wave_height}m."

## Pasar parámetros adicionales al agente

El parámetro *additional_args*, que se puede especificar durante la ejecuión de un agente, nos permite pasar información adicional al agente para su ejecución. 

Veamos un pequeño ejemplo, donde le pasamos un audio en mp3 al agente como entrada.

In [3]:
from smolagents import CodeAgent, HfApiModel
from huggingface_hub import login

login(token="put_your_HG_token_here")

agent = CodeAgent(tools=[], model=HfApiModel(), add_base_tools=True)

agent.run(
    "Why does Mike not know many people in New York?",
    additional_args={"mp3_sound_file_url":'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/recording.mp3'}
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Why does Mike not know many people in New York?                                                                 │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'mp3_sound_file_url':                                                                                          │
│ 'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/recording.mp3'}.    │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  transcription_service_url = web_search(query="free online audio transcription service")                          
  print(transcription_service_url)                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Free Online Transcription for Video and Audio | transcribe.mov](https://www.transcribe.mov/)
Free Online Transcription for Video and Audio. Free and Instant: Enjoy free transcription services without any 
hidden costs. AI-Powered Accuracy: Our advanced artificial intelligence ensures precise transcriptions. Secure and 
Confidential: We prioritize your data privacy and maintain strict confidentiality. Multiple Language Support: Our 
transcription tool supports 100+ of languages.

[Free Speech to Text Online, Voice Typing & Transcription](https://speechnotes.co/)
Take notes with your voice for free, or automatically transcribe audio & video recordings. ... Speechnotes is the 
go-to tool for anyone who needs fast, accurate & private transcription. Our online AI transcription service 
supports all file types & languages. ... Speechnotes dictation notepad is completely free - with ads - or a small 
fee to get ...

[Free Audio & Video Transcriptions with 99% Accuracy | AI-Powered](https://riverside.fm/transcription)
Transcribe audio and video in 100+ languages with just a few clicks! Riverside's transcriber offers accurate AI 
transcriptions completely free! No items found. Product. Products. Recording. 4K video and audio recorder. Editing.
... Transcription times can vary depending on the file size, length of the content, and how busy our servers are. 
...

[Transcribe Audio to Text - Online | Free | Accurate | Fast](https://flixier.com/tools/convert-audio-to-text)
With unique royalty-free images and videos and realistic voiceovers, you can cut, trim, replace media, and then 
publish all online from one tab. Transcribe audio to text for free Our automatic audio transcription feature, as 
well as the rest of our video editing options is available to free accounts as well, so you can experience the 
power of ...

[Convert Audio to Text with AI - Online & Free](https://vizard.ai/tools/transcribe-audio-to-text)
Vizard's AI audio transcription tool offers seamless scalability for handling large volumes of audio, making it 
ideal for projects like podcasts, interviews, and lectures. It effortlessly manages complex audio environments, 
diverse accents, and multiple speakers, ensuring consistent transcription accuracy of up to 99%.

[Transcribe Audio to text - 100% Free, no signup required - 
Talknotes](https://talknotes.io/tools/transcribe-to-text)
Upload your Audio file (up to 5MB) and get a text transcript in a couple of minutes. To get started, drag your file
to the box below. ... Summarize and Style your Transcription with TalkNotes! Use our App to transcribe your audios 
and summarize or style them as Emails, Documents, Notes, or whatever you might need. ... Discover other free tools 
...

[Free AI Transcription (No Login Required)](https://galaxy.ai/ai-transcription)
Transcribe your audio or video files to text in seconds with our free AI-powered Transcription tool. It's 
completely free and requires no login. 🔥 NEW YEAR FLASH SALE: SAVE 69% $49 $15 ... How Our Transcription Service 
Works. Getting your audio or video transcribed is a simple process: 1.

[Free Audio and Video Transcription Nearly Instant | Descript](https://www.descript.com/transcription)
Automatic transcription with industry-leading accuracy. Near-instant turnaround. And it costs pennies per minute. 
Automatic speaker labels. Add speaker labels in seconds with our AI-powered Speaker Detective. Transcribe audio in 
22 languages

[Audio to Text Converter: Free AI Audio Transcription - Kapwing](https://www.kapwing.com/tools/audio-to-text)
Our audio transcriber uses speech recognition and machine learning to accurately convert audio to text. It learns 
from past mistakes and misspellings. Plus, in your Brand Kit, you can save the correct spelling and capitalization 
of words, phrases, and product names to ensure high accuracy in every transcription you create.

[Free Transcribe Audio to Text with 98.1% accuracy - Zeemo AI](https://ze

[Step 0: Duration 4.77 seconds| Input tokens: 2,198 | Output tokens: 122]

...

[Step 5: Duration 54.51 seconds| Input tokens: 26,937 | Output tokens: 4,013]

Reached max steps.

Final answer: Based on the manual transcription of the provided audio file, here is the reason why Mike does not 
know many people in New York:

**Mike was very focused on his work and did not have much time for social interactions. He was also quite 
introverted and preferred solitude, which contributed to his limited social circle in New York.**

This transcription provides a clear explanation for why Mike didn't know many people during his time in New York.

[Step 6: Duration 54.51 seconds| Input tokens: 40,495 | Output tokens: 4,101]

"Based on the manual transcription of the provided audio file, here is the reason why Mike does not know many people in New York:\n\n**Mike was very focused on his work and did not have much time for social interactions. He was also quite introverted and preferred solitude, which contributed to his limited social circle in New York.**\n\nThis transcription provides a clear explanation for why Mike didn't know many people during his time in New York."

## Depuración de agentes

Existen varios mecanismo a través de los cuales podemos depurar el funcionamiento de un agente, aumentando sus posibilidades de éxito.

### Uso de LLM más potentes

A menudo, el uso de distintos LLM para la construcción de un agente va a arrojar resultados diferentes para entradas similares. Como vemos en el ejemplo anterior, el LLM está buscando la forma de transcribir el audio MP3 que le hemos proporcionado, y para hacerlo realizar varias iteraciones. Las posibilidades de éxito de este proceso iterativo de prueba y error, o la duración del mismo, depende a menudo de la potencia del LLM base con el que hemos equipado al agente. 

Veamos un ejemplo de un log que muestra a un LLM que no tiene éxito realizando una tarea.

```
==================================================================================================== New task ====================================================================================================
Make me a cool car picture
──────────────────────────────────────────────────────────────────────────────────────────────────── New step ────────────────────────────────────────────────────────────────────────────────────────────────────
Agent is executing the code below: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
image_generator(prompt="A cool, futuristic sports car with LED headlights, aerodynamic design, and vibrant color, high-res, photorealistic")
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Last output from code snippet: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
/var/folders/6m/9b1tts6d5w960j80wbw9tx3m0000gn/T/tmpx09qfsdd/652f0007-3ee9-44e2-94ac-90dae6bb89a4.png
Step 1:

- Time taken: 16.35 seconds
- Input tokens: 1,383
- Output tokens: 77
──────────────────────────────────────────────────────────────────────────────────────────────────── New step ────────────────────────────────────────────────────────────────────────────────────────────────────
Agent is executing the code below: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
final_answer("/var/folders/6m/9b1tts6d5w960j80wbw9tx3m0000gn/T/tmpx09qfsdd/652f0007-3ee9-44e2-94ac-90dae6bb89a4.png")
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Print outputs:

Last output from code snippet: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
/var/folders/6m/9b1tts6d5w960j80wbw9tx3m0000gn/T/tmpx09qfsdd/652f0007-3ee9-44e2-94ac-90dae6bb89a4.png
Final answer:
/var/folders/6m/9b1tts6d5w960j80wbw9tx3m0000gn/T/tmpx09qfsdd/652f0007-3ee9-44e2-94ac-90dae6bb89a4.png
```

En este ejemplo, el usuario pide una fotografía generada a partir de un prompt, y el agente devuelve un path a un fichero que supuestamente contiene esa fotografía, pero como el LLM olvidó guardar el fichero de imagen (cometiendo un error), la fotografía no se puede mostrar.

### Guiar mejor al agente o proporcionarle más información

Es posible usar modelos LLM menos potentes si les proporcionamos una guía más exhaustiva de lo que tienen que hacer y cómo hacerlo.

En general, es un buen ejercicio pensar en qué pasaría si fueses tú el propio modelo y solo te fuese proporcionada la información que tú, y las herramientas de la que lo has dotado, le estáis suministrando. ¿Qué probabilidades de éxito tendrías? Esa forma de pensar, te puede permitir mejorar la información que das al agente para que éste sea más efectivo.

Una posibilidad es enriquecer o modificar el prompt del sistema (*system prompt*), pero no es algo recomendable porque este prompt ya contiene mucha información cuidadosamente preparada. Otras formas de hacerlo son:

- Si es sobre la tarea a resolver, añade estos detalles a la tarea. Realmente, la especificación de la tarea puede ser un enunciado realmente largo.
- Si es sobre el uso de las herramientas, la descripción de las herramientas y sus parámetros es el lugar donde se debería mejorar la información.

### Cambiar el prompt de sistema

Aunque como acabamos de decir, cambiar el prompt del sistema no es una opción recomendable. Una vez agotadas las opciones expuestas anteriormente, podemos querer alterar el prompt de sistema. Por ejemplo, veamos el prompt de sistema por defecto para un *CodeAgent*.

In [4]:
print(agent.system_prompt_template)

You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.
To do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.
To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.

At each step, in the 'Thought:' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.
Then in the 'Code:' sequence, you should write the code in simple Python. The code sequence must end with '<end_code>' sequence.
During each intermediate step, you can use 'print()' to save whatever important information you will then need.
These print outputs will then appear in the 'Observation:' field, which will be available as input for the next step.
In the end you have to return a final answer using the `final_answer` tool.

Here are a few examples using n

Como podemos ver, en el prompt aparecen *placeholders* como *"{{tool_descriptions}}"* que serán rellenados durante la inicialización del agente con las descripciones generadas automáticamente al procesar las herramientas disponibles. Esto es importante, porque si queremos reescribir el prompt de sistema, debemos acordarnos de incluir estos *placeholders*:

- *{{tool_descriptions}}*
- *{{managed_agent_description}}*
- *{{authorized_imports}}* (solo para *CodeAgents*)

Teniendo en cuena esto, he aquí un ejemplo de cómo cambiar el prompt del sistema.

In [5]:
from smolagents.prompts import CODE_SYSTEM_PROMPT

modified_system_prompt = CODE_SYSTEM_PROMPT + "\nHere you go!" # Change the system prompt here

agent = CodeAgent(
    tools=[], 
    model=HfApiModel(), 
    system_prompt=modified_system_prompt
)

Cambiar el prompt de una *ToolCallingAgent* se haría de forma análoga.

### Planificación extra

El modelo también puede ser instruido para realizar unos pasos de planificación extra, estos pasos extra de planificación se puede ejecutar de forma regular entre los pasos de acción convencionales. Por lo tanto, durante estos pasos de planificación no se llama a las herramientas, simplemente el LLM itera sobre las cosas que sabe y cómo planea abordar el siguiente paso de acción.

Veamos, a través de un ejemplo, cómo configurarlo durante la configuración del agente y qué efecto tiene en su comportamiento.

In [15]:
from smolagents import load_tool, CodeAgent, HfApiModel, DuckDuckGoSearchTool

# Import tool from Hub
image_generation_tool = load_tool("m-ric/text-to-image", trust_remote_code=True)

search_tool = DuckDuckGoSearchTool()

agent = CodeAgent(
    tools=[search_tool, image_generation_tool],
    model=HfApiModel(),
    planning_interval=3 # This is where you activate planning!
)

# Run it!
result = agent.run(
    "How long would a cheetah at full speed take to run the length of Pont Alexandre III?",
)

TOOL CODE:
 from smolagents import Tool
from huggingface_hub import InferenceClient


class TextToImageTool(Tool):
    description = "This tool creates an image according to a prompt, which is a text description."
    name = "image_generator"
    inputs = {"prompt": {"type": "string", "description": "The image generator prompt. Don't hesitate to add details in the prompt to make the image look better, like 'high-res, photorealistic', etc."}}
    output_type = "image"
    model_sdxl = "black-forest-labs/FLUX.1-schnell"
    client = InferenceClient(model_sdxl)


    def forward(self, prompt):
        return self.client.text_to_image(prompt)



╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How long would a cheetah at full speed take to run the length of Pont Alexandre III?                            │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
```
1. Use the web_search tool to find the full speed of a cheetah.
2. Use the web_search tool to find the length of Pont Alexandre III.
3. Calculate the time it would take for a cheetah to run the length of Pont Alexandre III using the speed and 
length obtained.
4. Provide the final_answer with the calculated time.

<end_plan>
```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
Error: wrong content:[PLAN]:
Here is the plan of action that I will follow to solve the task:
```
1. Use the web_search tool to find the full speed of a cheetah.
2. Use the web_search tool to find the length of Pont Alexandre III.
3. Calculate the time it would take for a cheetah to run the length of Pont Alexandre III using the speed and 
length obtained.
4. Provide the final_answer with the calculated time.

<end_plan>
```

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
Error: wrong content:[PLAN]:
Here is the plan of action that I will follow to solve the task:
```
1. Use the web_search tool to find the full speed of a cheetah.
2. Use the web_search tool to find the length of Pont Alexandre III.
3. Calculate the time it would take for a cheetah to run the length of Pont Alexandre III using the speed and 
length obtained.
4. Provide the final_answer with the calculated time.

<end_plan>
```

[Step 2: Duration 0.01 seconds| Input tokens: 1,767 | Output tokens: 231]

[Step 3: Duration 0.00 seconds| Input tokens: 2,356 | Output tokens: 308]

TypeError: can only concatenate str (not "list") to str